<a href="https://colab.research.google.com/github/grugg1233/SemanticImageDetection/blob/main/ASSIGNMENT2_ATTALLAH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import zipfile
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir("/content/drive/MyDrive/datasets")
import zipfile
from pathlib import Path

def unzip(zip_path, out_dir):
    zip_path = Path(zip_path)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(out_dir)

unzip("/content/drive/MyDrive/datasets/carparts-seg.zip", "/content/carparts")
unzip("/content/drive/MyDrive/datasets/carparts.v2-indistro.yolov8.zip", "/content/hardcases")

print("Unzip done.")
print("carparts top:", os.listdir("/content/carparts")[:20])
print("hardcases top:", os.listdir("/content/hardcases")[:20])



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Unzip done.
carparts top: ['images', 'carparts-seg.yaml', 'labels']
hardcases top: ['train', 'README.roboflow.txt', 'README.dataset.txt', 'valid', 'data.yaml']


Unzip done.
carparts top: ['labels', 'carparts-seg.yaml', 'images']
hardcases top: ['valid', 'data.yaml', 'README.roboflow.txt', 'README.dataset.txt', 'train']


In [3]:
import shutil

IMG_EXTS = {".jpg", ".jpeg", ".png", ".webp"}

def ensure_dirs(root):
    root = Path(root)
    for split in ["train", "val"]:
        (root / "images" / split).mkdir(parents=True, exist_ok=True)
        (root / "labels" / split).mkdir(parents=True, exist_ok=True)

def copy_pairs(src_images, src_labels, dst_root, dst_split, prefix):
    src_images = Path(src_images)
    src_labels = Path(src_labels)
    dst_root = Path(dst_root)

    dst_images = dst_root / "images" / dst_split
    dst_labels = dst_root / "labels" / dst_split

    if not src_images.exists():
        raise FileNotFoundError(f"Missing images dir: {src_images}")
    if not src_labels.exists():
        raise FileNotFoundError(f"Missing labels dir: {src_labels}")

    n = 0
    for img in src_images.iterdir():
        if img.suffix.lower() not in IMG_EXTS:
            continue

        lab = src_labels / (img.stem + ".txt")

        # If label missing, create an empty label file (negative example)
        if not lab.exists():
            lab.write_text("")

        new_img = f"{prefix}__{img.name}"
        new_lab = f"{prefix}__{img.stem}.txt"

        shutil.copy2(img, dst_images / new_img)
        shutil.copy2(lab, dst_labels / new_lab)
        n += 1

    return n

merged_root = "/content/merged"
ensure_dirs(merged_root)

In [4]:
def find_first_existing(*candidates):
    for c in candidates:
        if Path(c).exists():
            return c
    return None

# carparts could be: /content/carparts/images/train OR /content/carparts/carparts-seg/images/train
carparts_train_images = find_first_existing(
    "/content/carparts/images/train",
    "/content/carparts/carparts-seg/images/train",
    "/content/carparts/carparts-seg/images/train"
)
carparts_train_labels = find_first_existing(
    "/content/carparts/labels/train",
    "/content/carparts/carparts-seg/labels/train"
)
carparts_val_images = find_first_existing(
    "/content/carparts/images/val",
    "/content/carparts/carparts-seg/images/val"
)
carparts_val_labels = find_first_existing(
    "/content/carparts/labels/val",
    "/content/carparts/carparts-seg/labels/val"
)

# hardcases could be: /content/hardcases/train/images OR /content/hardcases/<project>/train/images
hard_train_images = find_first_existing(
    "/content/hardcases/train/images",
    "/content/hardcases/carparts.v2-indistro.yolov8/train/images",
)
hard_train_labels = find_first_existing(
    "/content/hardcases/train/labels",
    "/content/hardcases/carparts.v2-indistro.yolov8/train/labels",
)
hard_valid_images = find_first_existing(
    "/content/hardcases/valid/images",
    "/content/hardcases/carparts.v2-indistro.yolov8/valid/images",
)
hard_valid_labels = find_first_existing(
    "/content/hardcases/valid/labels",
    "/content/hardcases/carparts.v2-indistro.yolov8/valid/labels",
)

print("Resolved paths:")
print("carparts_train_images:", carparts_train_images)
print("carparts_train_labels:", carparts_train_labels)
print("carparts_val_images  :", carparts_val_images)
print("carparts_val_labels  :", carparts_val_labels)
print("hard_train_images    :", hard_train_images)
print("hard_train_labels    :", hard_train_labels)
print("hard_valid_images    :", hard_valid_images)
print("hard_valid_labels    :", hard_valid_labels)


Resolved paths:
carparts_train_images: /content/carparts/images/train
carparts_train_labels: /content/carparts/labels/train
carparts_val_images  : /content/carparts/images/val
carparts_val_labels  : /content/carparts/labels/val
hard_train_images    : /content/hardcases/train/images
hard_train_labels    : /content/hardcases/train/labels
hard_valid_images    : /content/hardcases/valid/images
hard_valid_labels    : /content/hardcases/valid/labels


In [5]:
# Copy/merge
n1 = copy_pairs(carparts_train_images, carparts_train_labels, merged_root, "train", "carparts")
n2 = copy_pairs(carparts_val_images,   carparts_val_labels,   merged_root, "val",   "carparts")

n3 = copy_pairs(hard_train_images, hard_train_labels, merged_root, "train", "hard")
n4 = copy_pairs(hard_valid_images, hard_valid_labels, merged_root, "val",   "hard")

print("Copied:", {"carparts_train": n1, "carparts_val": n2, "hard_train": n3, "hard_val": n4})


Copied: {'carparts_train': 3156, 'carparts_val': 401, 'hard_train': 139, 'hard_val': 24}


In [6]:
yaml_text = """\
path: /content/merged
train: images/train
val: images/val

names:
  0: back_bumper
  1: back_door
  2: back_glass
  3: back_left_door
  4: back_left_light
  5: back_light
  6: back_right_door
  7: back_right_light
  8: front_bumper
  9: front_door
  10: front_glass
  11: front_left_door
  12: front_left_light
  13: front_light
  14: front_right_door
  15: front_right_light
  16: hood
  17: left_mirror
  18: object
  19: right_mirror
  20: tailgate
  21: trunk
  22: wheel
"""
Path("/content/data_merged.yaml").write_text(yaml_text)
print("Wrote /content/data_merged.yaml")


Wrote /content/data_merged.yaml


In [11]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/models/best.pt")
results = model.train(
    data="/content/data_merged.yaml",
    imgsz=640,
    epochs=60,
    batch=8,
    lr0=1e-4,
    lrf=1e-2,
    warmup_epochs=2,
    weight_decay=5e-4,
    cos_lr=True,
    patience=15,
    close_mosaic=10,
    fliplr=0.5,
    degrees=5.0,
    translate=0.08,
    scale=0.40,
    project="/content/drive/MyDrive/yolo_runs",
    name="merged_finetune",
)

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40441MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/data_merged.yaml, degrees=5.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/models/best.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=merged_finetune, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto

In [ ]:

modelp = YOLO(f"{model.trainer.save_dir}/weights/best.pt")  # load a fine-tuned model

prediction_results = modelp.predict("https://github.com/ultralytics/assets/releases/download/v0.0.0/carparts-image.jpg", save=True)

In [ ]:
# Load a model
modele = YOLO(f"{model.trainer.save_dir}/weights/best.pt")  # load a fine-tuned model

# Export the model
modele.export(format="onnx")

In [10]:
import os
print("Train images:", len(os.listdir("/content/merged/images/train")))
print("Train labels:", len(os.listdir("/content/merged/labels/train")))
print("Val images:", len(os.listdir("/content/merged/images/val")))
print("Val labels:", len(os.listdir("/content/merged/labels/val")))

Train images: 3295
Train labels: 3295
Val images: 425
Val labels: 425


In [8]:
!uv pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (NVIDIA A100-SXM4-40GB, 40441MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 42.1/235.7 GB disk)


In [9]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")



True
NVIDIA A100-SXM4-40GB


In [15]:
!cp -r /content/merged /content/drive/MyDrive/datasets/
!cp data_merged.yaml /content/drive/MyDrive/datasets/

^C
